# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [2]:
# all import statements needed for the project, for example:

import math
from math import radians, sin, cos, asin, sqrt

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import re
import numpy as np

In [ ]:
# any general notebook setup, like log formatting
import warnings
# setting ignore as a parameter
warnings.filterwarnings('ignore') 

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [32]:
def calculate_distance(from_coord, to_coord):
    # calculate the dstance between two coordinates in kilometers
    
    # get the statistics from the imported file 
    pickup_longtitude = from_coord['pickup_longitude'].map(radians)
    pickup_latitude = from_coord['pickup_latitude'].map(radians)
    dropoff_longitude = to_coord['dropoff_longitude'].map(radians)
    dropoff_latitude = to_coord['dropoff_latitude'].map(radians)
    
  
    Radius_of_earth = 6378.1
    diff_lon = pickup_longtitude - dropoff_longitude
    diff_lat = pickup_latitude - dropoff_latitude
    
    # calculate the result using following equations
    part_one = (diff_lat/2).map(sin)**2 + pickup_latitude.map(cos)*dropoff_latitude.map(cos)* (diff_lon/2).map(sin)**2
    part_two =  part_one.map(sqrt).map(asin)
    
    dist = 2*part_two*Radius_of_earth
    
    return dist

In [33]:
def add_distance_column(dataframe):
    # using the distance calculating function above to get a new column for the original dataframe
    from_coord = dataframe[['pickup_latitude', 'pickup_longitude']]
    to_coord = dataframe[['dropoff_latitude', 'dropoff_longitude']]
    dataframe['distance'] = calculate_distance(from_coord, to_coord)
    return dataframe

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [171]:
def find_taxi_csv_urls():
    # get all the URLs meet the requirements from the given website.
    
    # input the known website
    TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

    # get the links meet the requirements(for yellow taxi from 2009-01 to 2015-06)
    response = requests.get(TAXI_URL)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    # derive all yellow taxi files
    yellow_taxi = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    all_links = [a["href"] for a in yellow_taxi]

    # derive 2009-01 to 2015-06 records for yellow taxi
    pattern = re.compile(
    r"yellow_tripdata_2009-([0-9]{2})|yellow_tripdata_201([0-4])-([0-9]{2})|yellow_tripdata_2015-0([1-6]).parquet"
    )
    
    # create a empty list in order to store the links
    URLs = []

    # using for loop to see whether the URLs match the pattern
    # and add to the list created before if the pattern is matched
    for link in all_links:
        match = pattern.search(link)
        if match:
            URLs.append(match.string)
            
    return URLs

In [176]:
def get_and_clean_month_taxi_data(url):
    # given URLs to get clean monthly taxi data.
    dirty_taxi = pd.read_parquet(url)
    
    # in case records in 2009 and other years have different column names
    dirty_taxi= dirty_taxi.rename(columns = {"tpep_pickup_datetime": "pickup_datetime",
                                "tpep_dropoff_datetime": "dropoff_datetime",
                                "Trip_Pickup_DateTime": "pickup_datetime",
                                "Trip_Dropoff_DateTime": "dropoff_datetime",
                                "Passenger_Count": "passenger_count",
                                "Total_Amt": "total_amount",
                                "Start_Lon": "pickup_longitude",
                                "Start_Lat": "pickup_latitude",
                                "End_Lon": "dropoff_longitude",
                                "End_Lat": "dropoff_latitude"})
    
    # delete invalid data points with unknown passenger_count
    clean_taxi = dirty_taxi[(dirty_taxi["passenger_count"] >= 1)]
    
    # delete invalid data points with zero total amount 
    clean_taxi = clean_taxi[clean_taxi["total_amount"] != 0]
    
    # using the given taxi_zone file to understand the "PULocationID"  and "DOLocationID" column
    taxi_zone = gpd.read_file('taxi_zones.zip')
    taxi_zone = taxi_zone.to_crs(4326)
    taxi_zone["longitude"] = taxi_zone.centroid.x
    taxi_zone["latitude"] = taxi_zone.centroid.y
    
    #remove unecessary columns under different columns settings
    
    # if 'PULocationID' included in the column
    # for data after 2010
    if 'PULocationID' in clean_taxi.columns:
        # get only the columns needed
        data_taxi = clean_taxi[["pickup_datetime","dropoff_datetime"]]
        # Remove the unnecessary columns
        data_taxi['pickup_longitude'] = clean_taxi['PULocationID'].map(taxi_zone.centroid.x)
        data_taxi['pickup_latitude'] = clean_taxi['PULocationID'].map(taxi_zone.centroid.y)
        data_taxi['dropoff_longitude'] = clean_taxi['DOLocationID'].map(taxi_zone.centroid.x)
        data_taxi['dropoff_latitude'] = clean_taxi['DOLocationID'].map(taxi_zone.centroid.y)
    else:
        # for data before 2010
        # select the columns needed
        data_taxi = clean_taxi[['pickup_datetime','dropoff_datetime','pickup_longitude',
                           'pickup_latitude','dropoff_longitude','dropoff_latitude']]
    
    # drop invalid points with in selected columns
    data_taxi = data_taxi.dropna()
    
    # remove the points outside (40.560445, -74.242330) and (40.908524, -73.717047).
    data_taxi = data_taxi[(data_taxi['pickup_longitude'] > NEW_YORK_BOX_COORDS[0][1]) & 
                          (data_taxi['pickup_longitude'] < NEW_YORK_BOX_COORDS[1][1]) &
                          (data_taxi['pickup_latitude'] > NEW_YORK_BOX_COORDS[0][0]) &
                          (data_taxi['pickup_latitude'] < NEW_YORK_BOX_COORDS[1][0])]   
    
    # Sample monthly data in order to roughly get the same amount of data points 
    # over the given date range for both Taxi data and Uber data.
    # Given 78 months within the range, we need to sample out 200,000/78 data out of
    # the cleaned data
    sample_taxi = data_taxi.sample(n = 2565)
    sample_taxi.set_index(pd.Series(range(2565)), inplace=True)
    
    # Normalize column names
    taxi_data = sample_taxi.rename(columns = {"tpep_pickup_datetime": "pickup_datetime",
                                "tpep_dropoff_datetime": "dropoff_datetime"})
                     
    return taxi_data

In [178]:
def get_and_clean_taxi_data():
    # using for loop to get and clean data from every month through 2009-01 to 2015-06
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    # normalize the types of content in each column
    taxi_data[['pickup_datetime', 'dropoff_datetime']] = taxi_data[['pickup_datetime', 'dropoff_datetime']].apply(pd.to_datetime)
    taxi_data.iloc[:, 2:7] = taxi_data.iloc[:, 2:7].astype("float64")
    
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [24]:
def load_and_clean_uber_data(csv_file):
    uber = pd.read_csv(csv_file)
    
    # Remove invalid passenger count 
    uber = uber[(uber["passenger_count"]>=1)]
    
    #Pick useful columns
    uber_new = uber[["pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]]
    
    #Normalize the datetime
    uber_new["pickup_datetime"] = uber_new["pickup_datetime"].astype("datetime64[ns]")

    #Normalize the columns with float content
    uber_new.iloc[:, 1:5] = uber_new.iloc[:, 1:5].astype("float64")
    
    #Remove trips that start or end outside the designated coordinate box 
    #Make sure that the longtitude is in the box 
    uber_new = uber_new[(uber_new["pickup_longitude"]>=NEW_YORK_BOX_COORDS[0][1]) & (uber_new["pickup_longitude"]<=NEW_YORK_BOX_COORDS[1][1])]
    #Make sure that the latitude is in the box 
    uber_new = uber_new[(uber_new["pickup_latitude"]>=NEW_YORK_BOX_COORDS[0][0]) & (uber_new["pickup_longitude"]<=NEW_YORK_BOX_COORDS[1][0])]
    
    return uber_new 

In [34]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    weather_dirty = pd.read_csv(csv_file)
    
    # remove unecessary columns
    weather_clean = weather_dirty[["DATE", "HourlyPrecipitation", "HourlyWindSpeed"]]
    
    # normalize the datetime column
    weather_clean["DATE"] = weather_clean["DATE"].astype("datetime64[ns]")
    
    # normalize the hourlypercipitation column   
    # calculate the mean of hourly precipitation by skipping NaN values
    mean_hourlypercipitation = weather_clean["HourlyPrecipitation"].notna().mean()    
    # replace the "T" values by 0 and NaN values by mean(calculated above)
    weather_clean["HourlyPrecipitation"] = weather_clean["HourlyPrecipitation"].replace(["T", np.nan], [0, mean_hourlypercipitation])
    
    # normalize the hourlywindspeed column
    # calculate the mean of hourly windspeed by skipping NaN values
    mean_hourlywindspeed = weather_clean["HourlyWindSpeed"].notna().mean()    
    # replace NaN values by mean(calculated above)
    weather_clean["HourlyWindSpeed"] = weather_clean["HourlyWindSpeed"].replace(np.nan, mean_hourlywindspeed)
    
    # nomarlize the float columns
    weather_clean[["HourlyPrecipitation", "HourlyWindSpeed",]] = weather_clean[["HourlyPrecipitation", "HourlyWindSpeed",]].astype("float64")
    
    return weather_clean

In [ ]:
def clean_month_weather_data_daily(csv_file):
    weather_hourly = clean_month_weather_data_hourly(csv_file)
    weather_daily = pd.DataFrame().assign(Precipitation=weather_hourly['HourlyPrecipitation'], Windspeed=weather_hourly['HourlyWindSpeed'])
    weather_daily["DATE"] = weather_hourly["DATE"].dt.date
    weather_daily = weather_daily.groupby('DATE').agg({'Precipitation': np.mean, 'Windspeed': np.mean})
    weather_daily = weather_daily.reset_index()
    return weather_daily

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    weather_csv_files = ["2009_weather.csv", "2010_weather.csv", "2011_weather.csv",
                        "2012_weather.csv", "2013_weather.csv", "2014_weather.csv", "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [179]:
taxi_data = get_and_clean_taxi_data()

100%|███████████████████████████████████████████| 78/78 [53:24<00:00, 41.08s/it]


In [180]:
taxi_data

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-01-01 01:05:38,2015-01-01 01:15:07,-73.983025,40.714733,-73.915029,40.831417,14.197723
1,2015-01-16 10:35:02,2015-01-16 10:41:28,-73.915975,40.852521,-74.140788,40.628167,31.357264
2,2015-01-08 07:53:02,2015-01-08 07:59:10,-73.915975,40.852521,-73.912849,40.818260,3.823016
3,2015-01-04 02:10:12,2015-01-04 02:15:42,-73.962364,40.687967,-73.915029,40.831417,16.459822
4,2015-01-18 16:59:31,2015-01-18 17:07:09,-73.997380,40.728340,-73.915975,40.852521,15.432563
...,...,...,...,...,...,...,...
2560,2009-12-14 22:01:00,2009-12-14 22:07:00,-73.981380,40.747015,-73.983118,40.755507,0.956614
2561,2009-12-22 10:35:33,2009-12-22 10:42:09,-73.975060,40.761932,-73.985628,40.750859,1.521017
2562,2009-12-05 14:15:01,2009-12-05 14:25:30,-73.967088,40.788449,-73.970448,40.767699,2.327166
2563,2009-12-04 15:20:51,2009-12-04 15:28:18,-73.982723,40.745457,-73.999417,40.744205,1.414817


In [181]:
taxi_data.to_csv('taxi_data.csv')

In [ ]:
uber_data = get_uber_data()

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
        CREATE TABLE IF NOT EXISTS hourly_weather
        (
            id INTEGER PRIMARY KEY,
            DATE DATETIME,
            HourlyPrecipitation Float,
            HourlyWindSpeed FLOAT
            )
"""

DAILY_WEATHER_SCHEMA = """
         CREATE TABLE IF NOT EXISTS hourly_weather
        (
            id INTEGER PRIMARY KEY,
            DATE DATETIME,
            Precipitation Float,
            Windspeed FLOAT
            )
"""

TAXI_TRIPS_SCHEMA = """

        CREATE TABLE IF NOT EXISTS taxi_trips
        (
            id INTEGER PRIMARY KEY,
            pickup_datetime DATETIME,
            dropoff_datetime DATETIME,
            pickup_longtitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longtitude FLOAT,
            dropoff_latitude FLOAT,
            distance Float
            
        )
        
"""

UBER_TRIPS_SCHEMA = """
    CREATE TABLE IF NOT EXISTS taxi_trips
        (
            id INTEGER PRIMARY KEY,
            pickup_datetime DATETIME,
            pickup_longitude FLOAT,
            pickup_latitude FLOAT,
            dropoff_longtitude FLOAT,
            dropoff_latitude FLOAT
            distance Float
            
        )
"""

: 

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)